# URLs in Wayback SPN Data

In addition to looking at popular host names it also could be useful to identify popular URLs that people (or bots) archived on each day. Were there attempts to archive multiple things on the same day, and what can we possibly infer about the significance of these multiple attempts?

The trouble is that when a browser interacts with SavePageNow via the [web form](https://web.archive.org) it receive the HTML for the requested webpage which has been rewritten to include some JavaScript. This JavaScript gets the browser to request any additional resources that are needed for rendering the page (JavaScript, images, CSS, etc) through SavePageNow as well. This means that a more high-fidelity recording is made, since all the resources for a web page are needed to make it human readable.

Some of these URLs may be for things like jQuery a Content Deliver Network, or a CSS file. These aren't terribly interesting in terms of this analysis which is attempting to find duplicates in the originally requested page. One thing we can do is limit our analysis to HTML pages, or requests that come back 200 OK with a `Content-Type` HTTP header containing text/html.

In [2]:
import sys
sys.path.append('../utils')

from warc_spark import init, extractor

sc, sqlc = init()

In order to find the URLs it's important that we also retain the User-Agent that executed the request, since this tells us something about the person or agent who initiated SavePageNow. Unfortunately the User-Agent is in the WARC Request record, and the Content-Type of the response is in the WARC Response record. To complicate matters further SavePageNow may record a response using a *revist* record if the response is identical to a previously response. This can happen when a given URL is requested multiple times in specific time window. Luckily these three record types can be merged together using the WARC-Record-ID and the WARC-Concurrent-To WARC headers.

The `get_urls` function takes a WARC Record and depending on whether it is a request, response or revisit will return a tuple containing the record id and a dictionary with either a "ua" or "url" key (depending on the type of record). These dictionaries will be merged in the next step.

In [5]:
import re
from urllib.parse import urlparse

@extractor
def get_urls(record, warc_file):
    
    date = record.rec_headers.get_header('WARC-Date')
    
    if record.rec_type == 'request':
        id = record.rec_headers.get_header('WARC-Concurrent-To')
        ua = record.http_headers.get('user-agent')
        if id and ua:
            yield (id, {"ua": ua, "date": date})
            
    elif record.rec_type in ['response', 'revisit'] and 'html' in record.http_headers.get('content-type', ''):
        id = record.rec_headers.get_header('WARC-Record-ID')
        url = record.rec_headers.get_header('WARC-Target-URI')
        status_code = record.http_headers.get_statuscode()
        
        # not all 200 OK text/html responses are for requests for HTML 
        # for example some sites return 200 OK with some HTML when an image isn't found
        # this big of logic will try to identify known image, css and javascript extensions
        # to elmiminate them from consideration.
        
        uri = urlparse(url)        
        is_dependency = re.match(r'.*\.(gif|jpg|jpeg|js|png|css)$', uri.path)
        if not is_dependency and status_code == '200' and id and url:
            yield (id, {"url": url, "date": date, "warc_file": warc_file})

Now we can analyze our WARC data by selecting the WARC files we want to process and applying the `get_urls` function to them.

In [6]:
from glob import glob

warc_files = glob('warcs/*/*.warc.gz')
warcs = sc.parallelize(warc_files)
results = warcs.mapPartitions(get_urls)
results.take(5)

[('<urn:uuid:744cf8bb-4a90-4c78-9a05-34d63c37c1c6>',
  {'ua': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:63.0) Gecko/20100101 Firefox/63.0',
   'date': '2018-10-25T21:47:18Z'}),
 ('<urn:uuid:94cdf3c7-f437-4bb8-926e-292ced77030f>',
  {'ua': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36',
   'date': '2018-10-25T21:47:18Z'}),
 ('<urn:uuid:c8f70507-f78c-4cf6-a544-69bbb53f8ad2>',
  {'ua': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:61.0) Gecko/20100101 Firefox/61.0',
   'date': '2018-10-25T21:47:19Z'}),
 ('<urn:uuid:9d11abcf-b15d-47a2-8fcb-05adf194222a>',
  {'url': 'https://www.youtube.com/channel/UC6x7GwJxuoABSosgVXDYtTw',
   'date': '2018-10-25T21:47:18Z',
   'warc_file': 'warcs/liveweb-20181025224655/live-20181025214719-wwwb-app53.us.archive.org.warc.gz'}),
 ('<urn:uuid:9d11abcf-b15d-47a2-8fcb-05adf194222a>',
  {'ua': 'Wget/1.19.5 (linux-gnu)', 'date': '2018-10-25T21:47:18Z'})]

Now we can use [combineByKey](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/) method to merge the dictinaries using the WARC-Record-ID as a key.

In [7]:
def unpack(d1, d2):
    d1.update(d2)
    return d1

# merge the dataset using the record-id
dataset = results.combineByKey(
    lambda d: d,
    unpack,
    unpack
)

# filter out non-html requests (things without a url)
dataset = dataset.filter(lambda r: 'url' in r[1] and 'ua' in r[1])

dataset.take(5)

[('<urn:uuid:5fc7e893-f91a-4fe7-bf0b-0f35d14c2958>',
  {'url': 'https://www.youtube.com/channel/UCXZkaZ4f7d3-YuMyy76yS6A',
   'date': '2018-10-25T21:47:19Z',
   'warc_file': 'warcs/liveweb-20181025224655/live-20181025214719-wwwb-app53.us.archive.org.warc.gz',
   'ua': 'Wget/1.19.5 (linux-gnu)'}),
 ('<urn:uuid:24ea637b-05c6-4591-8bfe-4a964650d8fa>',
  {'url': 'https://www.youtube.com/channel/UCQIUhhcmXsu6cN6n3y9-Pww/about',
   'date': '2018-10-25T21:47:20Z',
   'warc_file': 'warcs/liveweb-20181025224655/live-20181025214719-wwwb-app53.us.archive.org.warc.gz',
   'ua': 'Wget/1.19.5 (linux-gnu)'}),
 ('<urn:uuid:a9d9d853-3ce0-4d6b-88ae-f42d713b60c6>',
  {'url': 'https://www.youtube.com/channel/UCQaX6TS_0hKrucF0LcuvWxA/about',
   'date': '2018-10-25T21:47:21Z',
   'warc_file': 'warcs/liveweb-20181025224655/live-20181025214719-wwwb-app53.us.archive.org.warc.gz',
   'ua': 'Wget/1.19.5 (linux-gnu)'}),
 ('<urn:uuid:9da6bcdf-122d-45df-8e27-1fad5cb8d98e>',
  {'url': 'https://www.youtube.com/channe

Finally we're going to convert our dictionaries into tuples so we can easily create a DataFrame out of them for analysis. As we don this we are also going to add two new columns for the User-Agent Family and whether it is a known bot. Some JSON files that were developed as part of the UserAgents notebook can help with this.

In [9]:
import json
ua_families = json.load(open('results/user-agent-families.json'))
top_uas = json.load(open('results/top-user-agents.json'))

def unpack(r):
    id = r[0]
    url = r[1].get("url", "")
    ua = r[1].get("ua", "")
    date = r[1].get("date", "")
    warc_file = r[1]["warc_file"]
    ua_f = ua_families.get(ua, "")
    bot = top_uas.get(ua_f, False)
    return (id, warc_file, date, url, ua, ua_f, bot)

unpacked_dataset = dataset.map(unpack)

# Convert to a Spark DataFrame
df = unpacked_dataset.toDF(["record_id", "warc_file", "date", "url", "user_agent", "user_agent_family", "bot"])

In [10]:
df.head(5)

[Row(record_id='<urn:uuid:5fc7e893-f91a-4fe7-bf0b-0f35d14c2958>', warc_file='warcs/liveweb-20181025224655/live-20181025214719-wwwb-app53.us.archive.org.warc.gz', date='2018-10-25T21:47:19Z', url='https://www.youtube.com/channel/UCXZkaZ4f7d3-YuMyy76yS6A', user_agent='Wget/1.19.5 (linux-gnu)', user_agent_family='Wget', bot=True),
 Row(record_id='<urn:uuid:24ea637b-05c6-4591-8bfe-4a964650d8fa>', warc_file='warcs/liveweb-20181025224655/live-20181025214719-wwwb-app53.us.archive.org.warc.gz', date='2018-10-25T21:47:20Z', url='https://www.youtube.com/channel/UCQIUhhcmXsu6cN6n3y9-Pww/about', user_agent='Wget/1.19.5 (linux-gnu)', user_agent_family='Wget', bot=True),
 Row(record_id='<urn:uuid:a9d9d853-3ce0-4d6b-88ae-f42d713b60c6>', warc_file='warcs/liveweb-20181025224655/live-20181025214719-wwwb-app53.us.archive.org.warc.gz', date='2018-10-25T21:47:21Z', url='https://www.youtube.com/channel/UCQaX6TS_0hKrucF0LcuvWxA/about', user_agent='Wget/1.19.5 (linux-gnu)', user_agent_family='Wget', bot=True)

This looks good, so let's save off these results before we do any more processing. We're going to use this dataset of URLs and their metadata in other notebooks.

In [12]:
from warc_spark import move_csv_parts

df.write.csv('results/urls', header=True, compression="gzip")

Now let's count the URLs by day and see which ones have appeared more than once. First we'll save these off as CSV.

In [20]:
df = sqlc.read.csv('results/urls', header=True)

from pyspark.sql.functions import countDistinct, desc, col

for year in range(2013, 2019):
    y = str(year)
    date = "{}-10-25".format(y)
    urls = df.filter(col('date').startswith(y))
    
    # useful in dev where note all data is being analyzed
    if urls.count() == 0:
        continue
        
    url_counts = urls.groupBy("url").count().sort(desc('count'))
    
    # remove the long tail of things that were only requested once
    url_counts = url_counts.filter(url_counts["count"] > 1)
    
    # flatten into a single csv
    url_counts = url_counts.coalesce(1)
    url_counts.write.csv('url-counts/{}'.format(date), header=True)
    
    # move 
    move_csv_parts('url-counts/{}'.format(date), 'results/{}/url-counts.csv'.format(date))

Let's see if it worked:

In [17]:
import pandas 

for year in range(2013, 2019):
    urls_by_year = pandas.read_csv('results/{}-10-25/url-counts.csv'.format(year))
    print()
    print(year)
    print(urls_by_year.head(10))


2013
                                                 url  count
0                            http://www.swydm.com/?f    218
1                        http://www.information.com/    174
2                         http://lifestream.aol.com/     94
3  http://assets.tumblr.com/analytics.html?f6cde1...     72
4  http://www.tumblr.com/assets/html/iframe/tease...     72
5  http://adf.send.microad.jp/afr.php?zoneid=6767...     72
6  http://assets.tumblr.com/assets/html/iframe/o....     70
7                              http://www.swydm.com/     63
8                                http://swydm.com/?f     54
9     http://www.stumbleupon.com/hostedbadge.php?s=5     43

2014
                                                 url  count
0  http://www.santeck.eu/mod_forum/login.php?acti...    388
1          http://www.santeck.eu/mod_forum/index.php    337
2               http://crazystorebolivia.com/?q=user    237
3  http://www.santeck.eu/mod_forum/profile.php?id...    210
4          https://smallbusi